# Setting up Spark

Start a pySpark session including Diana-Hep spark-root and histogrammar APIs

In [1]:
import pyspark.sql

#### CLUSTER CONFIG ####
# 5 Nodes
# 2 cores per Node
# 2.9 GB RAM per Node
#######################

num_cores = 1 # core per executer
num_executors = 5

session = pyspark.sql.SparkSession.builder \
    .master('mesos://10.64.22.90:5050') \
    .appName('DemoWithML') \
    .config('spark.jars.packages','org.diana-hep:spark-root_2.11:0.1.16,org.diana-hep:histogrammar-sparksql_2.11:1.0.4') \
    .config('spark.driver.extraClassPath','/opt/hadoop/share/hadoop/common/lib/EOSfs.jar') \
    .config('spark.executor.extraClassPath','/opt/hadoop/share/hadoop/common/lib/EOSfs.jar') \
    .config("spark.serializer","org.apache.spark.serializer.KryoSerializer") \
    .config('spark.cores.max','10') \
    .config('spark.executor.memory','2g') \
    .config('spark.executor.cores',`num_cores`) \
    .config('spark.executor.instances', `num_executors`) \
    .getOrCreate()
    
sqlContext = session
print sqlContext.version

print 'SparkSQL sesssion created'

2.2.1
SparkSQL sesssion created


# Loading root files stored remotely on EOS via xrootd

Loading root files (NanoAOD CMS format) from CERN public EOS area via xrootd.
Trees are read using the spark-root

In [2]:
from pyspark.sql.functions import lit
from samples import *

DFList = [] 

for s in samples:
    print 'Loading {0} sample from EOS file'.format(s) 
    dsPath = "root://eospublic.cern.ch//eos/opstest/cmspd-bigdata/"+samples[s]['filename']    
    tempDF = sqlContext.read \
                .format("org.dianahep.sparkroot") \
                .option("tree", "Events") \
                .load(dsPath) \
                .withColumn("pseudoweight", lit(samples[s]['weight']))\
                .withColumn("sample", lit(s))                
    DFList.append(tempDF)

Loading TT sample from EOS file
Loading WW sample from EOS file
Loading SingleMuon sample from EOS file
Loading ZZ sample from EOS file
Loading DYJetsToLL sample from EOS file
Loading WZ sample from EOS file


# Access DataFrame content

Get list of columns in the DataFrame ("branches" of the equivalent ROOT TTree).

In [3]:
#tempDF = sqlContext.read.format("org.dianahep.sparkroot.experimental").option("tree", "Events") \
#        .load("root://eospublic.cern.ch//eos/opstest/cmspd-bigdata/SingleMuonRun2016C-03Feb2017-v1.root")

#tempDF.printSchema()

# Data reduction

Subsets of interesting attributes can be selected via 'select' operations on the DataFrames (equivalent to "pruning" steps in ROOT-based frameworks).

All datasets can be joint into a single DataFrame (e.g. collecting data from various samples).

In [4]:
columns = [
    ### MUON
    'Muon_pt',
    'Muon_eta',
    'Muon_phi',
    'Muon_mass',
    'Muon_charge',
    'Muon_mediumId',
    'Muon_softId',
    'Muon_tightId',
    'nMuon',
    ### SAMPLE
    'sample',
    ### Jet
    'Jet_pt',
    'Jet_eta',
    'Jet_phi',
    'Jet_mass',
    'Jet_bReg',
    ### Weight
    'pseudoweight',
]

# Select columns from dataframe
DF = DFList[0].select(columns)
DF.printSchema()

# Merge all dataset into a single dataframe
for df_ in DFList[1:]:
    DF = DF.union(df_.select(columns))

print 'Partitions: {}'.format(DF.rdd.getNumPartitions()) ## this will show up 7 parallelize in JOB

root
 |-- Muon_pt: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- Muon_eta: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- Muon_phi: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- Muon_mass: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- Muon_charge: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- Muon_mediumId: array (nullable = true)
 |    |-- element: boolean (containsNull = true)
 |-- Muon_softId: array (nullable = true)
 |    |-- element: boolean (containsNull = true)
 |-- Muon_tightId: array (nullable = true)
 |    |-- element: boolean (containsNull = true)
 |-- nMuon: integer (nullable = true)
 |-- sample: string (nullable = false)
 |-- Jet_pt: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- Jet_eta: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- Jet_phi: array (nullable = true)


# Data selection

Selection of events based on features is obtained via a 'filter' operation.

Number of entries is obtained by 'count'.

In [5]:
#print 'total number of events in the DataFrame  = ', DF.count()
#print 'events in the DataFrame with \"nMuon > 0\" = ', DF.filter('nMuon > 0').count()

# Caching - 1

Dataframes can be cached into memory, shared across the Spark cluster nodes, for a faster access.

Caching takes some time...

In [6]:
DF.cache()

DataFrame[Muon_pt: array<float>, Muon_eta: array<float>, Muon_phi: array<float>, Muon_mass: array<float>, Muon_charge: array<int>, Muon_mediumId: array<boolean>, Muon_softId: array<boolean>, Muon_tightId: array<boolean>, nMuon: int, sample: string, Jet_pt: array<float>, Jet_eta: array<float>, Jet_phi: array<float>, Jet_mass: array<float>, Jet_bReg: array<float>, pseudoweight: double]

# Caching - 2

... but ensures fast data-handling operations afterwards.

In [ ]:
print 'total number of events in the DataFrame  = ', DF.count()
#print 'events in the DataFrame with \"nMuon > 0\" = ', DF.filter('nMuon > 0').count()
#DF['Jet_pt'][0]

total number of events in the DataFrame  = 

# Data inspection


Events can be inspected with 'show' (as in TTree.Show() ), also concatenating 'select' and 'filter'.

In [ ]:
#DF.filter(DF['sample'] == 'DYJetsToLL')\
#  .select('sample','nMuon','Muon_pt','Muon_eta','Muon_phi','Muon_charge','Jet_pt','Jet_eta','Jet_phi')\
#  .show(5)

# Create derivate quantities and structures - 1

User defined functions can be used for transformations evalueted row by row to compute derived quantity, such as invaraint mass of two physics objects involving multiple column.
The return value is added as a new column in the output DataFrame.

Dimuon candidate structure is created as an example.

In [ ]:
from math import *
from pyspark.sql.types import *

dimuonSchema = StructType([
    StructField("pass", BooleanType(), False),   # True if filled / False if default(empty) 
    #
    StructField("mass", FloatType(), False),     # Dimuon mass
    StructField("pt", FloatType(), False),       # Dimuon pt
    StructField("eta", FloatType(), False),      # Dimuon eta
    StructField("phi", FloatType(), False),      # Dimuon phi
    StructField("dPhi", FloatType(), False),     # DeltaPhi(mu1,mu2)
    StructField("dR", FloatType(), False),       # DeltaR(mu1,mu2)
    StructField("dEta", FloatType(), False),     # DeltaEta(mu1,mu2)
    #
    StructField("pt1", FloatType(), False),   # leading mu pT 
    StructField("pt2", FloatType(), False),   # sub-leading mu pT 
    StructField("eta1", FloatType(), False),  # leading mu eta
    StructField("eta2", FloatType(), False),  # sub-leading mu eta
    StructField("phi1", FloatType(), False),  # leading mu phi
    StructField("phi2", FloatType(), False),  # sub-leading mu phi
])

dijetSchema = StructType([
    StructField("mass", FloatType(), False),     # Dijet mass
    StructField("pt", FloatType(), False),       # Dijet pt
    StructField("eta", FloatType(), False),      # Dijet eta
    StructField("phi", FloatType(), False),      # Dijet phi
    StructField("dPhi", FloatType(), False),     # DeltaPhi(jet1,jet2)
    StructField("dR", FloatType(), False),       # DeltaR(jet1,jet2)
    StructField("dEta", FloatType(), False),     # DeltaEta(jet1,jet2)
])

#def binaryWeight(pt):
    #for idx in range(len(pt)):
    #    if pt[idx]>50:
    #        return 0.
    #    else:
    #        return 1.

def deltaPhi(phi1,phi2):
    ## Catch if being called with two objects
    if type(phi1) != float and type(phi1) != int:
        phi1 = phi1.phi
    if type(phi2) != float and type(phi2) != int:
        phi2 = phi2.phi
    ## Otherwise
    dphi = (phi1-phi2)
    while dphi >  pi: dphi -= 2*pi
    while dphi < -pi: dphi += 2*pi
    return dphi

def deltaR(eta1,phi1,eta2=None,phi2=None):
    ## catch if called with objects
    if eta2 == None:
        return deltaR(eta1.eta,eta1.phi,phi1.eta,phi1.phi)
    ## otherwise
    return hypot(eta1-eta2, deltaPhi(phi1,phi2))

def invMass(pt1, pt2, eta1, eta2, phi1, phi2, mass1, mass2):
    #
    theta1 = 2.0*atan(exp(-eta1))
    px1    = pt1 * cos(phi1)
    py1    = pt1 * sin(phi1)
    pz1    = pt1 / tan(theta1)
    E1     = sqrt(px1**2 + py1**2 + pz1**2 + mass1**2)

    theta2 = 2.0*atan(exp(-eta2))
    px2    = pt2 * cos(phi2)
    py2    = pt2 * sin(phi2)
    pz2    = pt2 / tan(theta2)
    E2     = sqrt(px2**2 + py2**2 + pz2**2 + mass2**2)

    themass  = sqrt((E1 + E2)**2 - (px1 + px2)**2 - (py1 + py2)**2 - (pz1 + pz2)**2)
    thept    = sqrt((px1 + px2)**2 + (py1 + py2)**2)
    thetheta = atan( thept / (pz1 + pz2) )        
    theeta   = 0.5*log( (sqrt((px1 + px2)**2 + (py1 + py2)**2 + (pz1 + pz2)**2)+(pz1 + pz2))/(sqrt((px1 + px2)**2 + (py1 + py2)**2 + (pz1 + pz2)**2)-(pz1 + pz2)) )
    thephi   = asin((py1 + py2)/thept)

    delPhi = deltaPhi(phi1,phi2)
    delR   = deltaR(eta1,phi1,eta2,phi2)
    delEta = eta1-eta2

    return (themass, thept, theeta, thephi, delPhi, delR, delEta)

def dimuonCandidate(pt, eta, phi, mass, charge, mediumid):
    # default class implementation   
    default_ = (False, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
    
    """
    Z->mm candidate from arbitrary muon selection:
      N(mu) >= 2
      pT > 30, 10
      abs(eta) < 2.4, 2.4
      mediumId muon
      opposite charge
    """
    
    if len(pt) < 2:
        return default_
    
    #Identify muon candidate
    leadingIdx = None
    trailingIdx = None
 
    for idx in range(len(pt)):
        if leadingIdx == None:
            if pt[idx] > 30 and abs(eta[idx]) < 2.4 and mediumid[idx]:
                leadingIdx = idx
        elif trailingIdx == None:
            if pt[idx] > 10 and abs(eta[idx]) < 2.4 and mediumid[idx]:
                trailingIdx = idx
        else:
            if pt[idx] > 10 and abs(eta[idx]) < 2.4 and mediumid[idx]:
                return default_

    if leadingIdx != None and trailingIdx != None and charge[leadingIdx] != charge[trailingIdx]:            
        # Candidate found
        dimuon_   = (True,) + \
                    invMass(pt[leadingIdx], pt[trailingIdx],
                            eta[leadingIdx], eta[trailingIdx],
                            phi[leadingIdx], phi[trailingIdx],
                            mass[leadingIdx], mass[trailingIdx]) + \
                    (pt[leadingIdx], pt[trailingIdx],
                     eta[leadingIdx], eta[trailingIdx],
                     phi[leadingIdx], phi[trailingIdx])
                
        return dimuon_
    else:
        return default_    

# Create derivate quantities and structures - 2

And a generic function filling the candidate structure can be defined.

# Create derivate quantities and structures - 3

Finally, a dimuon candidate structure can be appended to the DataFrame as an additional column

In [ ]:
from pyspark.sql.functions import udf
dimuonUDF = udf(dimuonCandidate, dimuonSchema)
DeltaRZJJUDF = udf(deltaR, FloatType())
dijetUDF = udf(invMass, dijetSchema)


#biweightUDF = udf(binaryWeight, FloatType())

DF = DF.withColumn('Dimuon', dimuonUDF ("Muon_pt",
                                        "Muon_eta",
                                        "Muon_phi",
                                        "Muon_mass",
                                        "Muon_charge",
                                        "Muon_mediumId")
                  )

DF = DF.withColumn('Dijet', dijetUDF ( DF["Jet_pt"][0],
                                       DF["Jet_pt"][1],
                                       DF["Jet_eta"][0],
                                       DF["Jet_eta"][1],
                                       DF["Jet_phi"][0],
                                       DF["Jet_phi"][1],
                                       DF["Jet_mass"][0],
                                       DF["Jet_mass"][1]
                                     )
                  )

#DF = DF.withColumn('pseudoweight', biweightUDF("Muon_pt"))

DF = DF.withColumn('DeltaRZjj', DeltaRZJJUDF( 'Dimuon.eta', 'Dimuon.phi', 'Dijet.eta', 'Dijet.phi' ) )

#DF.where('Dimuon.pass == True').select('Dimuon').show(3)
#DF.where('Dimuon.pass == True').select('pseudoweight').show(3)

# Caching 2

In [ ]:
#DF = DF.cache()

In [ ]:
#DF.where('Dimuon.pass == True').where('Muon_pt[0] > 40').select('pseudoweight','Dimuon').show(20)
#DF.where('Dimuon.pass == True').where('Muon_pt[0] > 40').select('pseudoweight','Dijet.mass').show(20)
#DF.where('Dimuon.pass == True').where('Muon_pt[0] > 40').select('pseudoweight','DeltaRZjj').show(20)

In [ ]:
DF.where('Dimuon.pass == True').where('Muon_pt[0] > 40').select('pseudoweight','Dijet.mass','Dijet.pt','Dijet.eta',
                                       'Dijet.phi','Dijet.dPhi','Dijet.dR','Dijet.dEta').show(20)
#DF.where('Dimuon.pass == True').where('Muon_pt[0] > 40').select('pseudoweight','Dijet.mass').show(20)

# Get statistics info about the data

Exploit pySparkSql functions to get statistical insights on the DataFrame

In [ ]:
from pyspark.sql.functions import *

#print 'Number of events, pre-selection level'

#DF.groupBy("sample").count().show()

#print 'Number of events, Dimuon invariant mass in [70-110] GeV'

#DF.where( (col("Dimuon.mass") > 70) & (col("Dimuon.mass") < 110) ).groupBy("sample").count().show()

#print 'Mean of Dimuon mass, evaluated in [70-110] GeV range'

#DF.where( (col("Dimuon.mass") > 70) & (col("Dimuon.mass") < 110) ).groupBy('sample').mean('Dimuon.mass').show()

#print 'Description of Dimuon mass features for SingleMuon dataset only, evaluated in [70-110] GeV range'

#DF.where( (col("Dimuon.mass") > 70) & (col("Dimuon.mass") < 110) & (DF["sample"] == "SingleMuon") ).describe('Dimuon.mass').show()
DF = DF.filter('Muon_pt[0] > 40').filter('Muon_pt[1] > 40')

# Caching 3

In [ ]:
#DF = DF.cache()

# Plotting the Zpeak mass

In [ ]:
# Load libraries, and append histogrammar functionalities to dataframe
import matplotlib.pyplot as plt
%matplotlib inline
import histogrammar as hg
import histogrammar.sparksql
import numpy as np

DF = DF.where( (col("Dimuon.mass") > 70) & (col("Dimuon.mass") < 110) )

hg.sparksql.addMethods(DF)

plots = hg.UntypedLabel(
    # 1d histograms
    LeadMuPt       = hg.Bin(20, 0, 500,    DF['Dimuon.pt1'], hg.Sum(DF['pseudoweight'])),
    LeadMuEta      = hg.Bin(48, -2.4, 2.4, DF['Dimuon.eta1'], hg.Sum(DF['pseudoweight'])),
    SubLeadMuPt    = hg.Bin(20, 0, 500,    DF['Dimuon.pt2'], hg.Sum(DF['pseudoweight'])),
    SubLeadMuEta   = hg.Bin(48, -2.4, 2.4, DF['Dimuon.eta2'], hg.Sum(DF['pseudoweight'])),
    ZInvMass      = hg.Bin(80, 70, 110,   DF['Dimuon.mass'], hg.Sum(DF['pseudoweight'])),
    ZDeltaR       = hg.Bin(20, 0, 3,      DF['Dimuon.dPhi'], hg.Sum(DF['pseudoweight'])),
    ZDeltaPhi     = hg.Bin(10, 0, 3,      DF['Dimuon.dR'], hg.Sum(DF['pseudoweight'])),
    
    LeadJetPt       = hg.Bin(20, 0, 500,    DF['Jet_pt'][0], hg.Sum(DF['pseudoweight'])),
    LeadJetEta      = hg.Bin(48, -2.4, 2.4, DF['Jet_eta'][0], hg.Sum(DF['pseudoweight'])),
    SubLeadJetPt    = hg.Bin(20, 0, 500,    DF['Jet_pt'][1], hg.Sum(DF['pseudoweight'])),
    SubLeadJetEta   = hg.Bin(48, -2.4, 2.4, DF['Jet_eta'][1], hg.Sum(DF['pseudoweight'])),
    JJInvMass      = hg.Bin(80, 70, 110,   DF['Dijet.mass'], hg.Sum(DF['pseudoweight'])),
    JJDeltaR       = hg.Bin(20, 0, 3,      DF['Dijet.dPhi'], hg.Sum(DF['pseudoweight'])),
    JJDeltaPhi     = hg.Bin(10, 0, 3,      DF['Dijet.dR'], hg.Sum(DF['pseudoweight'])),
    ZJJDeltaR       = hg.Bin(20, 0, 3,      DF['DeltaRZjj'], hg.Sum(DF['pseudoweight'])),
    
)

# Make a set of histograms, categorized per-sample
bulkHisto = hg.Categorize(quantity = DF['sample'], value = plots)

# Fill from spark
bulkHisto.fillsparksql(df=DF)


In [ ]:
for VARIABLE, num in zip(['ZInvMass','ZDeltaR','ZDeltaPhi','JJInvMass','JJDeltaR','JJDeltaPhi','ZJJDeltaR'],
                         ['331','332','333','334','335','336','337']
                        ):
    
    fig = plt.figure(num=None, figsize=(20, 10), dpi=80, facecolor='w', edgecolor='k')

    #aHisto   = bulkHisto("SingleMuon")(VARIABLE)
    #nBins    = len(aHisto.values)
    #edges    = np.linspace(aHisto.low, aHisto.high, nBins + 1)
    #width    = (aHisto.high - aHisto.low) / nBins

    plotVals = {}
    plt.subplot(num)
    
    for k in bulkHisto.bins:
        if k == 'SingleMuon':
            continue
            
        aHisto   = bulkHisto(k)(VARIABLE)
        nBins    = len(aHisto.values)
        edges    = np.linspace(aHisto.low, aHisto.high, nBins + 1)
        width    = (aHisto.high - aHisto.low) / nBins
            
        #plotVals[k] = [x.toJson()['data']*0.19 for x in bulkHisto(k)(VARIABLE).values]
        plotVals[k] = [x.sum for x in bulkHisto(k)(VARIABLE).values]
        plt.bar(edges[:-1], plotVals[k], width=width, label=k, color=samples[k]['color'], edgecolor=samples[k]['color'], fill=True)
    
    xdata   = np.linspace(aHisto.low+0.5*width, aHisto.high+0.5*width, nBins)    
    #ydata   = [x.toJson()['data'] for x in bulkHisto('SingleMuon')(VARIABLE).values]
    ydata   = [x.sum*4 for x in bulkHisto('SingleMuon')(VARIABLE).values]
    yerror  = [x**0.5 for x in ydata]

    plt.errorbar(xdata, ydata, fmt='ko', label="Data", xerr=width/2, yerr=yerror, ecolor='black')
    
    #if VARIABLE[-2:] == "Pt":
    #    plt.xlabel('Pt of Muon (GeV/c)')
    #elif VARIABLE[-4:] == "Mass":
    #    plt.xlabel('Dimuon invariant mass m($\mu\mu$) (GeV)')
    #else:
    #    plt.xlabel('Angular Unit')
    plt.xlabel('%s' %VARIABLE)
        
    plt.ylabel('Events / %s GeV' %width)
    plt.yscale('log')
    plt.legend(loc='upper right', fontsize='x-large', prop={'size': 10})
    

# ML Learning Extension

Preparing dataframe for machin learning. Here we define the set of features distinguishing the signal VH and backgrounds.

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.functions import rand
from pyspark.ml.linalg import Vectors, VectorUDT
#from pyspark.mllib.linalg import Vectors, VectorUDT
vectorizer = udf(lambda x: Vectors.dense(x), VectorUDT())

##Construct Desire Features:
#nFeatures = vectorizer(
#    array(
#        DF['Muon_Pt'][0],
#        DF['Muon_Eta'][0], 
#        DF['Jet_Pt'][0], 
#        DF['Jet_Eta'][0]
#    )
#) # 4 variables
nFeatures = vectorizer(
    array(
        'Dimuon.mass','Dimuon.dPhi','Dimuon.dR','Dijet.mass','Dijet.dPhi','Dijet.dR','DeltaRZjj'
    )
)


pseudodata = DF.withColumn( "features" , nFeatures ).select("features","sample")

#Split 
pseudodata = pseudodata.where( (col('sample') == "DYJetsToLL") | (col('sample') == "ZH") )
#Rename sample to Label
dataset = pseudodata.selectExpr("features as features", "sample as label").orderBy(rand())

In [ ]:
dataset.show(30)

## A quick run through on our cluster landscape

In [ ]:
#This variable is derived from the number of cores and executors, 
#and will be used to assign the number of model trainers.

num_workers = num_executors * num_cores

print("Number of desired executors: " + `num_executors`)
print("Number of desired cores / executor: " + `num_cores`)
print("Total number of workers: " + `num_workers`)

## Preprocessing dataframe

StringIndexer --> encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0

StandardScaler --> Apply feature normalization with standard scaling using Spark's StandardScaler. This will transform a feature to have mean 0, and std 1.

In [ ]:
from pyspark.ml.feature import StringIndexer

string_indexer = StringIndexer(inputCol="label", outputCol="index_label")
fitted_indexer = string_indexer.fit(dataset)
indexed_df = fitted_indexer.transform(dataset)

from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
fitted_scaler = scaler.fit(indexed_df)
scaled_df = fitted_scaler.transform(indexed_df)
print("The result of indexing and scaling. Each transformation adds new columns to the data frame:")
scaled_df.show(10)

After reading the dataset from storage, we will extract several metrics such as nb_features, which basically is the number of input neurons, and nb_classes, which is the number of classes (signal and background).

In [ ]:
nb_features = len(scaled_df.select("features").take(1)[0]["features"])
nb_classes = len(scaled_df.select("label").take(1)[0]["label"])
#nb_classes = 1

print("Number of features: " + str(nb_features))
print("Number of classes: " + str(nb_classes))

## Caching training set and test set in memory (WARNING)

plit up the dataset for training and testing purposes, and fetch some additional statistics on the number of training and testing instances.

In [ ]:
# Finally, we create a trainingset and a testset.
(training_set, test_set) = scaled_df.randomSplit([0.7, 0.3])
#training_set.cache()
#test_set.cache()

In [ ]:
training_set.show()
test_set.show()

## Distributing the training and test set to the workers

In [ ]:
test_set = test_set.repartition(num_workers)
training_set = training_set.repartition(num_workers)

##release DF
DF.unpersist()

training_set.cache()
test_set.cache()

num_test_set = test_set.count()
num_training_set = training_set.count()

print("Number of testset instances: " + str(num_test_set))
print("Number of trainingset instances: " + str(num_training_set))
print("Total number of instances: " + str(num_test_set + num_training_set))

test_set.show()

## Keras Model Construction

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
model = Sequential()
model.add(Dense(500, input_shape=(nb_features,)))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.6))
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.summary()

In [ ]:
optimizer = 'adagrad'
loss = 'categorical_crossentropy'

## Model Evaluation

In [ ]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from distkeras.transformers import LabelIndexTransformer
from distkeras.predictors import ModelPredictor
from distkeras.evaluators import *

from distkeras.trainers import SingleTrainer
#from distkeras.trainers import AEASGD
#from distkeras.trainers import DOWNPOUR

def evaluate(model):
    global test_set

    metric_name = "f1"
    evaluator = MulticlassClassificationEvaluator(metricName=metric_name, predictionCol="prediction_index", labelCol="index_label")
    # Clear the prediction column from the testset.
    test_set1 = test_set.select("scaled_features", "label", "index_label")
    # Apply a prediction from a trained model.
    predictor = ModelPredictor(keras_model=trained_model, features_col="scaled_features")
    test_set1 = predictor.predict(test_set1)
    # Transform the prediction vector to an indexed label.
    index_transformer = LabelIndexTransformer(output_dim=nb_classes)
    test_set1 = index_transformer.transform(test_set1)
    # Store the F1 score of the SingleTrainer.
    score = evaluator.evaluate(test_set1)
    
    return score

def evaluate_accuracy(model, test_set, features="scaled_features"):
    evaluator = AccuracyEvaluator(prediction_col="prediction_index", label_col="index_label")
    predictor = ModelPredictor(keras_model=trained_model, features_col=features)
    transformer = LabelIndexTransformer(output_dim=nb_classes)
    test_set2 = test_set.select(features, "index_label")
    test_set2 = predictor.predict(test_set2)
    test_set2 = transformer.transform(test_set2)
    score = evaluator.evaluate(test_set2)
    
    return score


In [ ]:
results = {}
time_spent = {}

## Model Training and evaluation: SingleTrainer

In [ ]:
trainer = SingleTrainer(keras_model=model, loss=loss, worker_optimizer=optimizer, 
                        features_col="scaled_features", num_epoch=1, batch_size=64)
trained_model = trainer.train(training_set)

In [ ]:
# Fetch the training time.
dt = trainer.get_training_time()
print("Time spent (SingleTrainer): " + `dt` + " seconds.")

# Evaluate the model.
score = evaluate(trained_model)
print("F1 (SingleTrainer): " + `score`)

# Evaluate Accuracy
accuracy = evaluate_accuracy(trained_model,test_set)
print("Accuracy: " + `accuracy`)

# Store the training metrics.
results['single'] = score
time_spent['single'] = dt

In [ ]:
test_set.show()

In [ ]:
test_set.printSchema()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import roc_curve, auc

